In [ ]:
# read pickle file and set up colab
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/My Drive/Dissertation'
from tqdm import tqdm
import pickle
import csv
import pandas as pd 

FPS_reviews = pd.read_csv('FPS_reviews_list.csv')

Mounted at /content/drive
/content/drive/My Drive/Dissertation


In [ ]:
# install and download packages for pre-processing

!pip install emot
!pip install emoji
!pip install emot.emo_unicode
!pip install gensim.parsing.preprocessing
import re
import pickle
from emot import *
import emoji
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
from emot.emo_unicode import EMOJI_UNICODE
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 18 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=d116e10268f9d829e5f32fe81b5fc3c07869f7f400790e7cc84380fb8cb2a649
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement emot.emo_unicode (from versions: none)
ERROR: No matching distribution found for emot.emo_unicode
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the

In [ ]:
# deal with profanities profanities censored as ♥. 
def identify_profanities_1(selected_reviews):
  with open("english-profanity.txt") as file:
    profanity_list=[]
    for item in file:
      profanity_list.append(item)
  profanity_dictionary={}
  for i in range(len(profanity_list)):
    word=profanity_list[i]
    censored="♥"*int(len(word))
# Create a dictionary which attempts to match the length of the profaned text with the actual text in the compiled list
    profanity_dictionary[word]=censored
  for i in tqdm(range(0, len(selected_reviews))):
    review=str(selected_reviews[i])
    for profanity in profanity_dictionary:
      review=review.replace(profanity_dictionary[profanity], profanity)
    selected_reviews[i]=review
  return (selected_reviews)

# deal with profanities censored as '*'
def identify_profanities_2(selected_reviews):
  with open("english-profanity.txt") as file:
    profanity_list=[]
    for item in file:
      profanity_list.append(item)
# Create a dictionary which attempts to match the length of the profaned text with the actual text in the compiled list
  profanity_dictionary={}
  for i in range(len(profanity_list)):
    word=profanity_list[i]
    censored="*"*int(len(word))
    profanity_dictionary[word]=censored
  for i in tqdm(range(0, len(selected_reviews))):
    review=str(selected_reviews[i])
    for profanity in profanity_dictionary:
      review=review.replace(profanity_dictionary[profanity], profanity)
    selected_reviews[i]=review
  return (selected_reviews)

In [ ]:
# deal with URLs

def remove_urls_https_and_www(selected_reviews):
  new_reviews_list=[]
  for i in tqdm(range(0, len(selected_reviews))):
    url=re.compile(r"url=https?://(www\.)?")
    selected_reviews[i] = url.sub('', str(selected_reviews[i])).strip().strip('/')
    url = re.compile(r"https?://(www\.)?")
    selected_reviews[i] = url.sub('', str(selected_reviews[i])).strip().strip('/')
    url = re.compile(r"www.?")
    selected_reviews[i] = url.sub('', str(selected_reviews[i])).strip().strip('/')
    url = re.compile(r".com\S+")
    selected_reviews[i] = url.sub('', str(selected_reviews[i])).strip().strip('/')
  return(selected_reviews)

# deal with emojis
def convert_emojis_to_word(selected_review):
  for i in tqdm(range(len(selected_review))):
    #print(type(selected_review[i]))
    review=str(selected_review[i])
    selected_review[i]=emoji.demojize(review, delimiters=("", " "))
  return selected_review

# deal with emoticons 
# adopted from https://stackoverflow.com/questions/70935501/unbalanced-parenthesis-regex
def convert_emoticons_to_word(selected_review):
  for i in tqdm(range(0,len(selected_review))):
    review=str(selected_review[i])
    for emot in EMOTICONS_EMO:
      review= review.replace(emot, EMOTICONS_EMO[emot].replace(" ","_"))
    selected_review[i]=review
  return selected_review

# deal with casing
def lower_case(selected_reviews):
  new_reviews_list=[]
  for i in tqdm(range(0, len(selected_reviews))):
    selected_reviews_lower = selected_reviews[i].lower()
    new_reviews_list.append(selected_reviews_lower)
  return(new_reviews_list)

# deal with punctuation
def remove_punctuation(selected_reviews):
  new_reviews_list=[]
  for i in tqdm(range(0, len(selected_reviews))):
    new_reviews = re.sub(r'[^A-Za-z0-9_]', ' ', str(selected_reviews[i]))
    new_reviews_list.append(new_reviews)
  return(new_reviews_list)


# deal with stop-words
def remove_stop_words(selected_reviews):
  with open('stopwords-english') as f:
    stopwords = f.read().splitlines()
  for i in tqdm(range(0,len(selected_reviews))):
    review=str(selected_reviews[i])
    selected_reviews[i]=remove_stopwords(review)
  return selected_reviews

# deal with double spacing 
def remove_spacing(selected_reviews):
  new_reviews_list=[]
  for i in tqdm(range(0, len(selected_reviews))):
    new_reviews = re.sub(r'\s+', ' ', str(selected_reviews[i]), flags=re.I)
    new_reviews_list.append(new_reviews)
  return(new_reviews_list)


In [ ]:
# run the above functions and save the file. 

FPS_reviews["review"]=identify_profanities_1(list(FPS_reviews["review"]))
FPS_reviews["review"]=identify_profanities_2(list(FPS_reviews["review"]))
FPS_reviews["review"]=remove_urls_https_and_www(list(FPS_reviews["review"]))
FPS_reviews["review"]=convert_emojis_to_word(list(FPS_reviews["review"]))
FPS_reviews["review"]=convert_emoticons_to_word(list(FPS_reviews["review"]))
FPS_reviews["review"]=lower_case(list(FPS_reviews["review"]))
FPS_reviews["review"]=remove_punctuation(list(FPS_reviews["review"]))
FPS_reviews["review"]=remove_stop_words(list(FPS_reviews["review"]))
FPS_reviews["review"]=remove_spacing(list(FPS_reviews["review"]))
FPS_reviews=FPS_reviews.drop(['Unnamed: 0'], axis=1)

FPS_reviews.to_csv('FPS_reviews_preprocessed.csv')

100%|██████████| 1056940/1056940 [00:11<00:00, 90317.27it/s]
